# Decoradores da bibilioteca padrão: `lru_cache`

In [21]:
from functools import lru_cache
from time import perf_counter

@lru_cache
def fibonacci(n: int) -> int:
    if n < 2: return n
    return fibonacci(n-1) + fibonacci(n-2)

In [22]:
N = 35

## Fibonacci recursivo do `N`-ésimo item (padrão)

In [23]:
_t0 = perf_counter()
for i in range(N):
    fibonacci(i)
dt = 1000 * (perf_counter() - _t0)
print('\n\tTempo: %.5f ms' %dt)


	Tempo: 0.05571 ms


## Fibonacci recursivo do `N`-ésimo item com cache

O cache foi implementado usando o decorator.

In [24]:
_t0 = perf_counter()
for i in range(N):
    fibonacci(i)
dt2 = 1000*(perf_counter() - _t0)
print('\n\tTempo: %.5f ms' %dt2)


	Tempo: 0.04667 ms


In [25]:
dt / dt2

1.1937599183641483

**Resultado**: com o cache, temos um código 22 mil vezes mais rápido.

# Decoradores da bibilioteca padrão: `singledispatch`

Temos a função `htmlize`

In [26]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"

htmlize([2,4,6])

'<pre>[2, 4, 6]</pre>'

mas queremos, agora, o seguinte comportamento:

`>>> htmlize([2, 'ok', True])`

```html
<ul>
  <li>2</li>
  <li><pre>ok</pre></li>
  <li><pre>True</pre></li>
</ul>
```

só que, em python, não há sobrecarga de funções.

In [27]:
from functools import singledispatch

O decorator `singledispatch` implementa uma forma de resolver isto.

Primeiro, decoramos a função geral.

In [28]:
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"

O processo de especialização ocorre como os dois exemplos abaixo (um pra texto e outro para número):

In [29]:
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return f"<p>{content}</p>"

In [30]:
import numbers

@htmlize.register(numbers.Real)
def _(n):
    return f"<pre class='number'>{n}</pre>"

In [31]:
@htmlize.register(bool)
def _(b):
    return f"<pre class='boolean'>{b}</pre>"

In [32]:
htmlize(19)

"<pre class='number'>19</pre>"

In [33]:
htmlize('Olá mundo!')

'<p>Olá mundo!</p>'

In [34]:
htmlize([4,3,5])

'<pre>[4, 3, 5]</pre>'

In [35]:
from collections.abc import MutableSequence


@htmlize.register(tuple)
@htmlize.register(MutableSequence)
def _(lista):
    content = "\n".join(f'<li>{htmlize(item)}</li>' for item in lista)
    return f"<ul>\n{content}\n</ul>"

In [20]:
print(htmlize([2, 'ok', True]))

<ul>
<li><pre class='number'>2</pre></li>
<li><p>ok</p></li>
<li><pre class='boolean'>True</pre></li>
</ul>


Legal, né?